In [1]:
from whisper_jax import FlaxWhisperPipline

pipeline = FlaxWhisperPipline("openai/whisper-large-v2")


/Users/shy/Projects/genad/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# JIT compile the forward call - slow, but we only do once
text = pipeline("audio.mp3", return_timestamps=True)

# used cached function thereafter - super fast!!
text = pipeline("audio.mp3", return_timestamps=True)

FileNotFoundError: [Errno 2] No such file or directory: 'audio.mp3'

In [ ]:
from pathlib import Path

# Get all files in the 'data' directory using pathlib
data_directory = Path('data')
mp4_files = [file for file in data_directory.glob('*.mp4')] + [file for file in data_directory.glob('*.MP4')]

print("MP4 files in 'data' directory:", mp4_files)



MP4 files in 'data' directory: [PosixPath('data/SettingSlopeGutter1b.mp4'), PosixPath('data/SettingSlopeGutter2.mp4')]


In [9]:
import subprocess

# Function to extract mp3 from mp4
def extract_mp3_from_mp4(mp4_file_path, output_directory):
    output_mp3_path = output_directory / f"{mp4_file_path.stem}.mp3"
    command = f"ffmpeg -i {mp4_file_path} {output_mp3_path}"
    subprocess.run(command, shell=True)
    return output_mp3_path

# Create a directory for the extracted mp3 files if it doesn't exist
mp3_directory = data_directory / 'mp3'
mp3_directory.mkdir(exist_ok=True)

# Extract mp3 files from each mp4 file
extracted_mp3_files = [extract_mp3_from_mp4(mp4_file, mp3_directory) for mp4_file in mp4_files]
print("Extracted MP3 files:", extracted_mp3_files)


Extracted MP3 files: [PosixPath('data/mp3/SettingSlopeGutter1b.mp3'), PosixPath('data/mp3/SettingSlopeGutter2.mp3')]


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopen

In [ ]:
# Transcribe each MP3 file using the Whisper pipeline with timestamps and save the transcripts
def transcribe_and_save(mp3_file_path, output_directory):
    # Transcribe the audio file with timestamps
    transcription_result = pipeline(str(mp3_file_path), return_timestamps=True)
    
    # Extract text from the transcription result
    transcription_text = "\n".join([f"{segment['start']} - {segment['end']}: {segment['text']}" for segment in transcription_result['segments']])
    
    # Define the output path for the transcript
    output_transcript_path = output_directory / f"{mp3_file_path.stem}.txt"
    
    # Write the transcription to a file
    with open(output_transcript_path, 'w') as file:
        file.write(transcription_text)
    
    return output_transcript_path

# Create a directory for the transcripts if it doesn't exist
transcript_directory = data_directory / 'transcripts'
transcript_directory.mkdir(exist_ok=True)

# Transcribe each MP3 file and save the transcripts with timestamps
transcribed_files = [transcribe_and_save(mp3_file, transcript_directory) for mp3_file in extracted_mp3_files]
print("Transcribed files saved with timestamps:", transcribed_files)



In [ ]:
import os

# Load OpenAI API key from environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
print("OpenAI API key loaded successfully.")


In [ ]:
import openai
import json

# Load OpenAI API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

def select_storyboard_segments(transcripts, goal):
    # Define the prompt for selecting relevant segments
    prompt_template = "Given the goal to '{}', select relevant segments from the following transcript: {}"
    
    # Initialize a dictionary to store filenames and their relevant segments
    storyboard_segments = {}
    
    # Iterate over each transcript file
    for transcript_file in transcripts:
        # Read the transcript content
        with open(transcript_file, 'r') as file:
            transcript_content = file.read()
        
        # Create the prompt for GPT
        prompt = f"Given the goal to '{goal}', select relevant segments from the following transcript: {transcript_content}"
        
        # Get the response from GPT-4
        response = openai.Completion.create(
            engine="gpt-4o",
            prompt=prompt,
            max_tokens=150,
            n=1,
            stop=None,
            temperature=0.5
        )
        
        # Extract the text from the response
        relevant_segments = response.choices[0].text.strip()
        
        # Store the relevant segments in the dictionary
        storyboard_segments[transcript_file] = relevant_segments
    
    return storyboard_segments

# Define the high level goal
high_level_goal = "make an advertising video"

# Select relevant segments for the storyboard
storyboard = select_storyboard_segments(transcribed_files, high_level_goal)
print("Storyboard segments selected:", json.dumps(storyboard, indent=4))



In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def create_storyboard_video(storyboard, original_video_path):
    # Load the original video
    original_clip = VideoFileClip(original_video_path)
    
    # List to hold all the clips to concatenate
    clips = []
    
    # Iterate over the storyboard segments
    for start_time, end_time in storyboard.values():
        # Extract the clip from the original video based on the start and end times
        clip = original_clip.subclip(start_time, end_time)
        clips.append(clip)
    
    # Concatenate all the clips into one final video
    final_clip = concatenate_videoclips(clips)
    
    # Write the result to a file
    output_path = "final_storyboard_video.mp4"
    final_clip.write_videofile(output_path, codec="libx264")
    
    return output_path

# Assuming 'original_video_path' is defined and contains the path to the original video
final_video_path = create_storyboard_video(storyboard, original_video_path)
print("Final storyboard video created at:", final_video_path)
